In [ ]:
import os

import pandas as pd

from adversarial import RandomAdversarialAlignments
from utils import load_config


config = load_config()

languages = ['en']

In [ ]:
if not os.path.exists('data/atis/adversarial'):
    os.mkdir('data/atis/adversarial')

for base_language in languages:
    other_languages = list(config['languages'])
    other_languages.remove(base_language)

    dataset = pd.DataFrame()

    for language in other_languages:
        dataset = pd.concat(
            (
                dataset,
                RandomAdversarialAlignments(
                    base_language=base_language,
                    attack_language=language,
                    subset='train',
                    num_examples=1
                ).generate_dataset('train')
            )
        )

    dataset.to_csv(f'data/atis/adversarial/adversarial_{base_language}.csv')